# Archivo a archivo
Este archivo tiene el propósito de proporcionar el estudio de cada archivo existente dentro del código Rust del paquete de go3 para explicar su funcionamiento clave, asimismo, se toma la iniciativa de presentar dos razones fundamentales para determinar que una comparación con PyGoSemSim no se considera viable.

## Código 1 - libs.rs
Este es el archivo intermedio que permite la comunicación entre Python y Rust, por ende, ya se deduce que la implementación de las funcionalidades de este paquete son completamente hechas en Rust que es envuelto por el paquete pyo3 que permite invocarlas bajo la sintaxis del lenguaje de destino. A continuación el desglose linea a linea.

In [ ]:
// En esta sección se realiza la envoltura de los archivos Rust con pyo3 estableciendolo en
// 3 pasos:
use pyo3::prelude::*;           //Rust comprende Python (viceversa).
use pyo3::types::PyModule;      //Rust entiende el concepto de módulo de Python.
use pyo3::wrap_pyfunction;      //Python invoca código de Rust.

// Se solicita la carga de tres archivo existentes en el mismo directorio en el que se
// encuentra el archivo actual.
pub mod go_loader;              // Lectura de archivos de entrada, relaciones e IC.
pub mod go_ontology;            // --
pub mod go_semantic;            // --

// Dentro de cada uno de estos archivos mencionados previamente, se solicita la disponibilidad
// de las funciones que se mencionan a continuación.
use go_loader::{load_go_terms, load_gaf, build_term_counter};
use go_ontology::{get_term_by_id, ancestors, common_ancestor, deepest_common_ancestor};
use go_semantic::{term_ic, semantic_similarity, batch_similarity, compare_genes, compare_gene_pairs_batch, set_num_threads};

// En esta sección se realiza la envoltura de todas las componentes invocadas anteriormente
// de forma en que estas sean reconocidad como modulo de Python.
#[pymodule]
fn go3(_py: Python, m: &Bound<'_, PyModule>) -> PyResult<()> {

    // Funciones añadidas por archivos.
    m.add_function(wrap_pyfunction!(load_go_terms, m)?)?;
    m.add_function(wrap_pyfunction!(load_gaf, m)?)?;
    m.add_function(wrap_pyfunction!(build_term_counter, m)?)?;

    m.add_function(wrap_pyfunction!(get_term_by_id, m)?)?;
    m.add_function(wrap_pyfunction!(ancestors, m)?)?;
    m.add_function(wrap_pyfunction!(common_ancestor, m)?)?;
    m.add_function(wrap_pyfunction!(deepest_common_ancestor, m)?)?;

    m.add_function(wrap_pyfunction!(set_num_threads, m)?)?;
    m.add_function(wrap_pyfunction!(term_ic, m)?)?;
    m.add_function(wrap_pyfunction!(semantic_similarity, m)?)?;
    m.add_function(wrap_pyfunction!(batch_similarity, m)?)?;
    m.add_function(wrap_pyfunction!(compare_genes, m)?)?;
    m.add_function(wrap_pyfunction!(compare_gene_pairs_batch, m)?)?;

    // Objetos o clases por archivo.
    m.add_class::<go_ontology::PyGOTerm>()?;
    m.add_class::<go_loader::GAFAnnotation>()?;
    m.add_class::<go_loader::TermCounter>()?;

    // Retroalimentación que indica exito o error en la migración de funciones.
    Ok(())
}

## Código 2 - goloader.rs
Archivo encargado del procesamiento de los archivos .obo y gaf para la construcción de las relaciones jerarquicas entre terminos, relaciones gen a termino y el cálculo de IC de cada término existente.

In [ ]:
// ---------------- Librerias.
use pyo3::prelude::*;                                               // Rust comprende Python (viceversa).
use once_cell::sync::OnceCell;                                      // Implementación de variables globales de una sola carga.
use rustc_hash::{FxHashMap as HashMap, FxHashSet as HashSet};       // Invocación de tablas hash.
use std::io::{BufReader, BufRead};                                  // Lectura eficiente de archivos.
use std::fs::File;                                                  // Apertura y manejo de archivos.
use parking_lot::RwLock;                                            // Administración de hilos por semaforo.
use std::path::Path;                                                // Administración de rutas.
use std::fs;                                                        // Administración de entradas y salidas.
use reqwest::blocking::get;                                         // Peticiones para obtención de recursos en internet.
use rayon::prelude::*;                                              // Automatización del uso de todos los nucleos disponibles de un dispositivo.

// ---------------- Componentes globales.
// Declaración de entidades estáticas en memoria y global en toda la ejecución del código;
// cabe señalar que estos son persistentes mientras el código se ejecuta y estos poseen la
// capacidad de ser leidos por varios threads con bloqueo si se requiere escribir.
use crate::go_ontology::{GOTerm, PyGOTerm, collect_ancestors, get_terms_or_error};
pub static GO_TERMS_CACHE: OnceCell<RwLock<HashMap<String, GOTerm>>> = OnceCell::new();                 // Hash entre el GO_ID y su información.
pub static GENE2GO_CACHE: OnceCell<RwLock<HashMap<String, Vec<String>>>> = OnceCell::new();             // Hash entre un gen y sus términos GO relacionados.
pub static ANCESTORS_CACHE: OnceCell<RwLock<HashMap<String, HashSet<String>>>> = OnceCell::new();       // Hash entre un GO_ID y sus respectivos ancestros.
pub static DCA_CACHE: OnceCell<RwLock<HashMap<(String, String), String>>> = OnceCell::new();            // DCA - Deep Common Ancestor entre un par de terminos.

/// Struct representing a single annotation from a GAF file.
/// 
/// Fields
/// ------
/// db_object_id : str
///   The gene product identifier (e.g., UniProt ID).
/// go_term : str
///   The GO term ID (e.g., GO:0008150).
/// evidence : str
///   The evidence code for the annotation (e.g., IEA).
/*De este componente se puede comprender que solamente del GAF se obtiene el db_object_id
el go_term y la evidencia asociada a este mismo, siendo las tres columnas claves.*/
#[pyclass]                      // Se considera una clase compatible con Python.
#[derive(Clone)]                // Permite replicación/copia si es requerido.
pub struct GAFAnnotation {
    #[pyo3(get)]
    pub db_object_id: String,
    #[pyo3(get)]
    pub go_term: String,
    #[pyo3(get)]
    pub evidence: String,
}

/// Struct holding annotation counts and information content (IC) for GO terms.
///
/// Fields
/// ------
/// counts : dict
///   Mapping from GO term ID to annotation count.
/// total_by_ns : dict
///   Mapping from namespace to total annotation count.
/// ic : dict
///   Mapping from GO term ID to information content (IC).
/*Estructura encargada de establecer la computación de information content (IC) de
cada termino GO asociado.*/
#[pyclass]
#[derive(Clone)]
pub struct TermCounter {
    #[pyo3(get)]
    pub counts: HashMap<String, usize>,         // term_id -> count
    #[pyo3(get)]
    pub total_by_ns: HashMap<String, usize>,    // namespace -> total annotations
    #[pyo3(get)]
    pub ic: HashMap<String, f64>,               // term_id -> IC
}

/// Parse a GO OBO file and return a map of GO term IDs to GOTerm structs.
///
/// Arguments
/// ---------
/// path : str
///   Path to the OBO file.
///
/// Returns
/// -------
/// dict
///   Map of GO term IDs to term structs.
pub fn parse_obo(path: &str) -> HashMap<String, GOTerm> {
    let contents = fs::read_to_string(path).expect("Can't open OBO file");                          // Carga completa en memoria del archivo
    let chunks = contents.split("[Term]");                                                          // Separación de chunks de líneas según campo [Term]

    let canonical_terms: Vec<GOTerm> = chunks                                                       // Procesamiento paralelo de GoTerms (ignora obsoletos o mal formateados).
        .par_bridge()
        .filter_map(parse_term_chunk)
        .filter(|term| !term.is_obsolete)
        .collect();

    let mut term_map: HashMap<String, GOTerm> =                                                     // Preparación de memoria y configurar uso de hashing rápido.
        HashMap::with_capacity_and_hasher(canonical_terms.len() * 2, Default::default());

    for term in canonical_terms.into_iter() {                                                       // Uso de procesamiento de otros identificadores y términos asociados a un
        // Collect the "synonym group": canonical + alts                                            // GoTerm de forma en que todos apunten a la misma información.
        let mut all_ids = term.alt_ids.clone();
        all_ids.push(term.id.clone());

        // Build GOTerm copies for each ID
        for id in &all_ids {
            let mut clone = term.clone();
            clone.id = id.clone();
            clone.alt_ids = all_ids.clone(); // all point to the same group
            term_map.insert(id.clone(), clone);
        }
    }

    // Compute children, levels, depths, etc.
    compute_levels_and_depths(&mut term_map);                                                       // Se verifica profundidad/especifidad de cada término.

    // Precompute ancestors for caching
    let ancestors_map: HashMap<String, HashSet<String>> = term_map                                  // Se identifican (una sola vez) los ancestros a cada GoTerm.
        .par_iter()
        .map(|(id, _)| {
            let ancestors = crate::go_ontology::collect_ancestors(id, &term_map)
                .into_iter()
                .map(|s| s.to_string())
                .collect();
            (id.clone(), ancestors)
        })
        .collect();
    let _ = ANCESTORS_CACHE.set(RwLock::new(ancestors_map));                                        // Guardado en cache.

    // Initialize DCA_CACHE
    let _ = DCA_CACHE.set(RwLock::new(HashMap::default()));                                         // Se inicializa cache de ancestro más cercano (se calcula por demanda).

    term_map                                                                                        // Se retorna mapeo de GoTermns.
}

/// Parse a chunk of text representing a single GO term from an OBO file.
///
/// Arguments
/// ---------
/// chunk : str
///   Text chunk for a single term.
///
/// Returns
/// -------
/// Option[GOTerm]
///   Parsed term or None if invalid/obsolete.
fn parse_term_chunk(chunk: &str) -> Option<GOTerm> {
    // Definición de la estructura de un término (dentro del archivo).
    let mut term = GOTerm {
        id: String::new(),                              // Identificador.
        name: String::new(),                            // Nombre.
        namespace: String::new(),                       // Ontología BP, MC, CC.
        definition: String::new(),                      // Definición.
        parents: Vec::new(),                            // Padres.
        is_obsolete: false,                             // ¿Es obsoleto?
        alt_ids: Vec::new(),                            // Ids alternativos.
        replaced_by: None,                              // Reemplazado por... (si esta obsoleto).
        consider: Vec::new(),                           // Considerar ¿Qué cosa?.
        synonyms: Vec::new(),                           // Sinónimos.
        xrefs: Vec::new(),                              // Referencias asociadas a...
        relationships: Vec::new(),                      // Relaciones con otros términois.
        comment: None,                                  // Comentarios.
        children: Vec::new(),                           // Hijos.
        level: None,                                    // Nivel.
        depth: None,                                    // Profundidad/Especificidad.
    };

    let chunk = chunk.split("[Typedef]").next().unwrap_or(chunk);       // Ignorar campos Typedef.

    let lines: Vec<&str> = chunk
        .lines()
        .map(|l| l.trim())                               // eliminamos espacios a izquierda y derecha
        .filter(|l| !l.is_empty())                       // quitamos líneas vacías
        .collect();

    if lines.is_empty() {
        return None;
    }
    let mut valid = false;

    // Verificación de campos que se obtienen de cada sección existente en Term.
    for line in lines {
        if line.starts_with("id: ") {                                               // Verificación para que sea válido (debe tener campo ID).
            term.id = line["id: ".len()..].to_string();
            valid = true;
        } else if line.starts_with("name: ") {
            term.name = line["name: ".len()..].to_string();
        } else if line.starts_with("namespace: ") {
            term.namespace = line["namespace: ".len()..].to_string();
        } else if line.starts_with("def: ") {
            term.definition = line["def: ".len()..].to_string();
        } else if line.starts_with("is_a: ") {
            let parent = line["is_a: ".len()..]
                .split_whitespace()
                .next()
                .unwrap_or("")
                .to_string();
            if !parent.is_empty() {
                term.parents.push(parent);
            }
        } else if line.starts_with("alt_id: ") {
            term.alt_ids.push(line["alt_id: ".len()..].to_string());
        } else if line.starts_with("is_obsolete: true") {
            term.is_obsolete = true;
        } else if line.starts_with("replaced_by: ") {
            term.replaced_by = Some(line["replaced_by: ".len()..].to_string());
        } else if line.starts_with("consider: ") {
            term.consider.push(line["consider: ".len()..].to_string());
        } else if line.starts_with("synonym: ") {
            term.synonyms.push(line["synonym: ".len()..].to_string());
        } else if line.starts_with("xref: ") {
            term.xrefs.push(line["xref: ".len()..].to_string());
        } else if line.starts_with("relationship: ") {
            let rel_def = &line["relationship: ".len()..];
            let mut parts = rel_def.split_whitespace();
            if let (Some(rel), Some(target)) = (parts.next(), parts.next()) {
                term.relationships.push((rel.to_string(), target.to_string()));
            }
        }
    }

    if valid {
        Some(term)                                       // Si el termino es válido lo devuelve, en caso contrario se ignora.
    } else {
        None
    }
}

/// Compute the level and depth for each GO term in the ontology.
///
/// Arguments
/// ---------
/// terms : dict
///   Mutable map of GO terms.
///
/// Returns
/// -------
/// None
///   Updates the `level` and `depth` fields of each term in-place.
pub fn compute_levels_and_depths(terms: &mut HashMap<String, GOTerm>) {
    
    // Paso 1: construir mapa de hijos teniendo en consideración las relaciones
    // is_a para establecer jerarquía padre-hijo.
    let mut child_map: HashMap<String, Vec<String>> = HashMap::default();
    for (id, term) in terms.iter() {
        for parent in &term.parents {
            child_map.entry(parent.clone()).or_default().push(id.clone());
        }
    }

    // Paso 2: inicializar level como una componente que identifica la distancia mínima
    // entre el término raiz y el termino en estudio. Esto como función recursiva interna
    // que selecciona camino según los padres de menor distancia.
    fn init_level(
        term_id: &str,
        terms: &mut HashMap<String, GOTerm>,
        visiting: &mut HashSet<String>,
    ) -> usize {
        if visiting.contains(term_id) {
            // Ciclo detectado: se evita recursión infinita
            eprintln!("⚠️ Ciclo detectado en level: {}", term_id);
            return 0;
        }

        if let Some(level) = terms.get(term_id).and_then(|t| t.level) {
            return level;
        }

        visiting.insert(term_id.to_string());

        let parents = terms
            .get(term_id)
            .map(|t| t.parents.clone())
            .unwrap_or_default();

        let level = if parents.is_empty() {
            0
        } else {
            parents
                .iter()
                .map(|p| init_level(p, terms, visiting))
                .min()
                .unwrap_or(0) + 1
        };

        visiting.remove(term_id);
        if let Some(term) = terms.get_mut(term_id) {
            term.level = Some(level);
        }

        level
    }

    // Paso 3: inicializar depth como la distancia más larga a la raíz. Mismo concepto 
    // que punto anterior.
    fn init_depth(
        term_id: &str,
        terms: &mut HashMap<String, GOTerm>,
        visiting: &mut HashSet<String>,
    ) -> usize {
        if visiting.contains(term_id) {
            eprintln!("Ciclo detectado en depth: {}", term_id);
            return 0;
        }

        if let Some(depth) = terms.get(term_id).and_then(|t| t.depth) {
            return depth;
        }

        visiting.insert(term_id.to_string());

        let parents = terms
            .get(term_id)
            .map(|t| t.parents.clone())
            .unwrap_or_default();

        let depth = if parents.is_empty() {
            0
        } else {
            parents
                .iter()
                .map(|p| init_depth(p, terms, visiting))
                .max()
                .unwrap_or(0) + 1
        };

        visiting.remove(term_id);
        if let Some(term) = terms.get_mut(term_id) {
            term.depth = Some(depth);
        }

        depth
    }

    // Paso 4: recorrer todos los términos y calcular level + depth como llamados
    // a las funciones por cada ids.
    let ids: Vec<String> = terms.keys().cloned().collect();
    for id in &ids {
        let mut visiting = HashSet::default();
        init_level(id, terms, &mut visiting);

        let mut visiting = HashSet::default();
        init_depth(id, terms, &mut visiting);
    }

    // Paso 5: rellenar el campo children con los hijos (solo vía is_a).
    for (parent, children) in child_map {
        if let Some(term) = terms.get_mut(&parent) {
            term.children = children;
        }
    }
}

/// Download the latest GO OBO file if not present locally.
///
/// Arguments
/// ---------
/// None
///
/// Returns
/// -------
/// str
///   Path to the downloaded or existing OBO file.
// Obtención de archivo .obo verificando si este existe en la ruta de destino, por defecto,
// descarga la versión básica ¿Por qué se tomo esa decisión?.
pub fn download_obo() -> Result<String, String> {
    let obo_path = "go-basic.obo";
    if Path::new(obo_path).exists() {
        return Ok(obo_path.to_string());
    }

    let url = "http://purl.obolibrary.org/obo/go/go-basic.obo";
    println!("Descargando ontología desde: {}", url);
    let response = get(url).map_err(|e| e.to_string())?;

    let content = response.text().map_err(|e| e.to_string())?;
    fs::write(obo_path, content).map_err(|e| e.to_string())?;

    Ok(obo_path.to_string())
}

/// Load GO terms from an OBO file and cache them globally.
///
/// Arguments
/// ---------
/// path : Optional[str]
///   Optional path to the OBO file.
///
/// Returns
/// -------
/// list of PyGOTerm
///   List of GO terms as Python objects.
// En esta función se realiza el llamado al parseo de GoTerms y su guardado en chache
// para no necesitar re-ejecuciones de la función.
#[pyfunction]
#[pyo3(signature = (path=None))]
pub fn load_go_terms(path: Option<String>) -> PyResult<Vec<PyGOTerm>> {
    let path = path.unwrap_or_else(|| download_obo().unwrap());
    let terms_map = parse_obo(&path);

    // Guardar en la caché global
    let _ = GO_TERMS_CACHE.set(RwLock::new(terms_map.clone()));

    // Devolver lista de PyGOTerm (compatibilidad con Python).
    let terms_vec = terms_map
        .into_iter()
        .map(|(_, v)| PyGOTerm::from(&v))
        .collect();

    Ok(terms_vec)
}

/// Load a GAF annotation file and cache the gene-to-GO mapping.
///
/// Arguments
/// ---------
/// path : str
///   Path to the GAF file.
///
/// Returns
/// -------
/// list of GAFAnnotation
///   List of parsed GAF annotations.
// 
#[pyfunction]
pub fn load_gaf(path: String) -> PyResult<Vec<GAFAnnotation>> {
    // Se verifica acceso al archivo .gaf existente, en caso de que este no pueda ser 
    // accedido, entonces la función termina.
    let file = File::open(&path)
        .map_err(|e| pyo3::exceptions::PyIOError::new_err(e.to_string()))?;
    let reader = BufReader::new(file);

    // Get the loaded GO terms to check for obsolete terms.
    // Recordar: Debieron haberse cargado en cache los go terms.
    let terms = match crate::go_ontology::get_terms_or_error() {
        Ok(t) => t,
        Err(e) => return Err(e),
    };

    let mut annotations: Vec<GAFAnnotation> = Vec::new();                   // Vector de anotaciones de GAF (parte vacía).
    let mut gene2go: HashMap<String, Vec<String>> = HashMap::default();     // Relación de genes a términos.

    // Lectura linea a linea filtrando aquellas que no empiecen con '!'.
    for line in reader.lines().filter_map(Result::ok).filter(|l| !l.starts_with('!')) {
        let cols: Vec<&str> = line.split('\t').collect();                   // Identificar columnas mediante una separación por tabuladores (son 7).
        if cols.len() < 7 {                                                 // Debe tener como mínimo 7 campos como mínimo.
            continue;
        }

        // Columnas consideradas de cada linea válida.
        let db_object_id = cols[1].to_string();
        let qualifier = cols[3].to_string();
        let mut go_term = cols[4].to_string();
        let evidence = cols[6].to_string();
        let gene = cols[2].to_string();

        // Se ignoran registros de genes con terminos sin evidencia sustentada,
        // asimismo de asociaciones del tipo "El gen NO hace esto".
        // Filter out ND annotations.
        if evidence == "ND" {
            continue;
        }
        // Skip NOT annotations.
        if qualifier.contains("NOT") {
            continue;
        }


        // Resolve obsolete terms
        if let Some(term) = terms.get(&go_term) {
            if term.is_obsolete {
                if let Some(ref replacement) = term.replaced_by {
                    // Use the replacement term instead
                    go_term = replacement.clone();
                } else if !term.consider.is_empty() {
                    // If there are "consider" terms, choose the first one
                    go_term = term.consider[0].clone();
                } else {
                    // If no replacement, drop the annotation
                    continue;
                }
            }
        } else {
            // GO term not found at all
            continue;
        }

        // Add annotation.
        annotations.push(GAFAnnotation {
            db_object_id: db_object_id.clone(),
            go_term: go_term.clone(),
            evidence,
        });

        // Update gene -> GO mapping
        gene2go.entry(gene).or_default().push(go_term);
    }

    // Save in global cache.
    let _ = GENE2GO_CACHE.set(RwLock::new(gene2go));

    Ok(annotations)
}
/// Build a term counter (counts, IC) from GAF annotations.
///
/// Arguments
/// ---------
/// py_annotations : list of GAFAnnotation
///   List of GAFAnnotation Python objects.
///
/// Returns
/// -------
/// TermCounter
///   Struct with counts and IC values.

#[pyfunction]
pub fn build_term_counter(
    py: Python<'_>,
    py_annotations: Vec<Py<GAFAnnotation>>,
) -> PyResult<TermCounter> {
    // Obtener los términos GO desde el caché global.
    let terms = get_terms_or_error()?;

    // Convertir las anotaciones de Py<GAFAnnotation> a GAFAnnotation (Rust).
    let annotations: Vec<GAFAnnotation> = py_annotations
        .into_iter()
        .map(|py_ann| py_ann.extract(py))
        .collect::<PyResult<_>>()?;

    // Llamar a la función de conteo interna.
    Ok(_build_term_counter(&annotations, &terms))
}

/// Internal: Build a term counter from Rust GAFAnnotation and GOTerm.
///
/// Arguments
/// ---------
/// annotations : list of GAFAnnotation
///   List of GAFAnnotation structs.
/// terms : dict
///   Map of GO terms.
///
/// Returns
/// -------
/// TermCounter
///   Struct with counts and IC values.
fn _build_term_counter(
    annotations: &[GAFAnnotation],
    terms: &HashMap<String, GOTerm>,
) -> TermCounter {
    use rayon::prelude::*;                          // Motor de paralelismo.
    use std::sync::Mutex;                           // Controlador de exclusión mutua.

    // Parallel: build obj_to_terms - Objeto con toda la relación entre los términos y las anotaciones
    // procesadas en el GAF.
    let obj_to_terms: HashMap<&str, HashSet<String>> = {
        let obj_to_terms_mutex: Mutex<HashMap<&str, HashSet<String>>> = Mutex::new(HashMap::default());
        annotations.par_iter().for_each(|ann| {
            let go_id = ann.go_term.as_str();
            let mut term_set: HashSet<String> = collect_ancestors(go_id, terms);
            term_set.insert(go_id.to_string());
            let mut map = obj_to_terms_mutex.lock().unwrap();
            map.entry(ann.db_object_id.as_str())
                .or_default()
                .extend(term_set);
        });
        obj_to_terms_mutex.into_inner().unwrap()
    };

    // Parallel: build counts and total_by_ns - Conteo de cada gen presente por termino y
    // cantidad de genes únicos por ontología.
    let (counts, total_by_ns) = {
        let counts_mutex: Mutex<HashMap<String, usize>> = Mutex::new(HashMap::default());
        let total_by_ns_mutex: Mutex<HashMap<String, usize>> = Mutex::new(HashMap::default());
        obj_to_terms.values().collect::<Vec<_>>().par_iter().for_each(|term_ids| {
            let mut namespaces_seen = HashSet::default();
            for term_id in *term_ids {
                if let Some(term) = terms.get(term_id.as_str()) {
                    let mut counts = counts_mutex.lock().unwrap();
                    *counts.entry(term_id.to_string()).or_insert(0) += 1;
                    namespaces_seen.insert(term.namespace.as_str());
                }
            }
            for ns in namespaces_seen {
                let mut total_by_ns = total_by_ns_mutex.lock().unwrap();
                *total_by_ns.entry(ns.to_string()).or_insert(0) += 1;
            }
        });
        (counts_mutex.into_inner().unwrap(), total_by_ns_mutex.into_inner().unwrap())
    };

    // Calcular IC (secuencial, as it's fast).
    let mut ic: HashMap<String, f64> = HashMap::default();
    for (term_id, count) in &counts {
        if let Some(term) = terms.get(term_id.as_str()) {
            let total = total_by_ns.get(&term.namespace).copied().unwrap_or(1);
            let freq = *count as f64 / total as f64;
            let info_content = if freq > 0.0 { -freq.ln() } else { 0.0 };
            ic.insert(term_id.clone(), info_content);
        }
    }

    TermCounter {
        counts,
        total_by_ns,
        ic,
    }
}

## Código 3 - go_ontology.rs


In [ ]:
// ---------------- Librerias.
use rustc_hash::{FxHashMap as HashMap, FxHashSet as HashSet};       // Invocación de tablas hash.      
use pyo3::prelude::*;                                               // Rust comprende Python (viceversa).
use pyo3::types::PyString;                                          // Se invoca tipo de dato string de Python.
use crate::go_loader::{GO_TERMS_CACHE, GENE2GO_CACHE};              // Se obtiene la cache generada en go_loader (si existe).
use pyo3::exceptions::PyValueError;                                 // Se invoca 'error por valor' de Python.

/// Struct representing a Gene Ontology (GO) term.
///
/// Fields
/// ------
/// id : str
///   GO term identifier (e.g., GO:0006397).
/// name : str
///   Human-readable name of the term.
/// namespace : str
///   Ontology namespace (e.g., biological_process).
/// definition : str
///   Textual definition of the term.
/// parents : list of str
///   List of parent GO term IDs (is_a relationships).
/// children : list of str
///   List of child GO term IDs (is_a relationships).
/// depth : Optional[int]
///   Maximum distance to a root term (None if not computed).
/// level : Optional[int]
///   Minimum distance to a root term (None if not computed).
/// is_obsolete : bool
///   True if the term is obsolete.
/// alt_ids : list of str
///   Alternative GO IDs for this term.
/// replaced_by : Optional[str]
///   If obsolete, the term that replaces this one.
/// consider : list of str
///   Suggested replacement terms if obsolete.
/// synonyms : list of str
///   List of synonyms.
/// xrefs : list of str
///   Cross-references to other databases.
/// relationships : list of (str, str)
///   Other relationships (e.g., part_of).
/// comment : Optional[str]
///   Additional comments.
#[derive(Clone)]
pub struct GOTerm {
    pub id: String,
    pub name: String,
    pub namespace: String,
    pub definition: String,
    pub parents: Vec<String>,
    pub children: Vec<String>,
    pub depth: Option<usize>,
    pub level: Option<usize>,
    pub is_obsolete: bool,
    pub alt_ids: Vec<String>,
    pub replaced_by: Option<String>,
    pub consider: Vec<String>,
    pub synonyms: Vec<String>,
    pub xrefs: Vec<String>,
    pub relationships: Vec<(String, String)>,
    pub comment: Option<String>,
}

/// Python-exposed struct representing a GO term (for use in the Python API).
///
/// Fields
/// ------
/// id : str
///   GO term identifier.
/// name : str
///   Human-readable name of the term.
/// namespace : str
///   Ontology namespace.
/// definition : str
///   Textual definition of the term.
/// parents : list of str
///   List of parent GO term IDs.
/// children : list of str
///   List of child GO term IDs.
/// depth : Optional[int]
///   Maximum distance to a root term.
/// level : Optional[int]
///   Minimum distance to a root term.
/// is_obsolete : bool
///   True if the term is obsolete.
/// alt_ids : list of str
///   Alternative GO IDs for this term.
/// replaced_by : Optional[str]
///   If obsolete, the term that replaces this one.
/// consider : list of str
///   Suggested replacement terms if obsolete.
/// synonyms : list of str
///   List of synonyms.
/// xrefs : list of str
///   Cross-references to other databases.
/// relationships : list of (str, str)
///   Other relationships (e.g., part_of).
/// comment : Optional[str]
///   Additional comments.
#[pyclass]
#[derive(Clone)]
pub struct PyGOTerm {
    #[pyo3(get)] pub id: String,
    #[pyo3(get)] pub name: String,
    #[pyo3(get)] pub namespace: String,
    #[pyo3(get)] pub definition: String,
    #[pyo3(get)] pub parents: Vec<String>,
    #[pyo3(get)] pub children: Vec<String>,
    #[pyo3(get)] pub depth: Option<usize>,
    #[pyo3(get)] pub level: Option<usize>,
    #[pyo3(get)] pub is_obsolete: bool,
    #[pyo3(get)] pub alt_ids: Vec<String>,
    #[pyo3(get)] pub replaced_by: Option<String>,
    #[pyo3(get)] pub consider: Vec<String>,
    #[pyo3(get)] pub synonyms: Vec<String>,
    #[pyo3(get)] pub xrefs: Vec<String>,
    #[pyo3(get)] pub relationships: Vec<(String, String)>,
    #[pyo3(get)] pub comment: Option<String>,
}

// Esta sección de código define como se debe realizar la copia de los
// datos de la estructura GoTerm de Rust a su versión de Python de forma
// en que no existan errores de compatibilidad
impl From<&GOTerm> for PyGOTerm {
    fn from(term: &GOTerm) -> Self {
        Self {
            id: term.id.clone(),
            name: term.name.clone(),
            namespace: term.namespace.clone(),
            definition: term.definition.clone(),
            parents: term.parents.clone(),
            children: term.children.clone(),
            depth: term.depth,
            level: term.level,
            is_obsolete: term.is_obsolete,
            alt_ids: term.alt_ids.clone(),
            replaced_by: term.replaced_by.clone(),
            consider: term.consider.clone(),
            synonyms: term.synonyms.clone(),
            xrefs: term.xrefs.clone(),
            relationships: term.relationships.clone(),
            comment: term.comment.clone(),
        }
    }
}

// Esta sección de código define el comportamiento de Python a imprimir
// un término GO como un mensaje que considera los elementos definidos
// previamente. Además de automatizar la escritura del nombre del
// objeto.
#[pymethods]
impl PyGOTerm {
    fn __repr__(slf: &Bound<'_, Self>) -> PyResult<String> {
        let class_name: Bound<'_, PyString> = slf.get_type().qualname()?;
        let s = slf.borrow();
        Ok(format!(
            "{} id: {}\nname: {}\nnamespace: {}\ndefinition: {}\nparents: {:?}\nchildren: {:?}\ndepth: {:?}\nlevel: {:?}\nis_obsolete: {}\nalt_ids: {:?}\nreplaced_by: {:?}\nconsider: {:?}\nsynonyms: {:?}\nxrefs: {:?}\nrelationships: {:?}\ncomments: {:?}",
            class_name, s.id, s.name, s.namespace, s.definition, s.parents, s.children, s.depth, s.level,
            s.is_obsolete, s.alt_ids, s.replaced_by, s.consider, s.synonyms, s.xrefs, s.relationships, s.comment
        ))
    }
}

// Esta sección genera un ciclo de regreso de la versión Python a la versión
// de Rust, esto hace que cualquier modificación de una capa afecte a otra.
impl From<PyGOTerm> for GOTerm {
    fn from(py_term: PyGOTerm) -> Self {
        Self {
            id: py_term.id,
            name: py_term.name,
            namespace: py_term.namespace,
            definition: py_term.definition,
            parents: py_term.parents,
            children: py_term.children,
            depth: py_term.depth,
            level: py_term.level,
            is_obsolete: py_term.is_obsolete,
            alt_ids: py_term.alt_ids,
            replaced_by: py_term.replaced_by,
            consider: py_term.consider,
            synonyms: py_term.synonyms,
            xrefs: py_term.xrefs,
            relationships: py_term.relationships,
            comment: py_term.comment,
        }
    }
}

// Las posteriores 2 secciones verifican si es posible acceder al cache de los
// Go terms guardados y la relación de genes a terminos GO, estableciendo un
// error RUNTIMEERROR en caso de no encontrarlos.

/// Get a read lock on the global GO terms map, or error if not loaded.
///
/// Arguments
/// ---------
/// None
///
/// Returns
/// -------
/// RwLockReadGuard<HashMap<String, GOTerm>>
///   Read guard for the GO terms map.
pub fn get_terms_or_error<'a>() -> PyResult<parking_lot::RwLockReadGuard<'a, HashMap<String, GOTerm>>> {
    Ok(
        GO_TERMS_CACHE
            .get()
            .ok_or_else(|| pyo3::exceptions::PyRuntimeError::new_err("GO terms not loaded. Call go3.load_go_terms() first."))?
            .read()
    )
}

/// Get a read lock on the global gene-to-GO mapping, or error if not loaded.
///
/// Arguments
/// ---------
/// None
///
/// Returns
/// -------
/// RwLockReadGuard<HashMap<String, Vec<String>>>
///   Read guard for the gene2go map.
pub fn get_gene2go_or_error<'a>() -> PyResult<parking_lot::RwLockReadGuard<'a, HashMap<String, Vec<String>>>> {
    Ok(
        GENE2GO_CACHE
            .get()
            .ok_or_else(|| pyo3::exceptions::PyRuntimeError::new_err("Gene2GO mapping not loaded. Call go3.load_gene2go() first."))?
            .read()
    )
}


/// Get the PyGOTerm object for a given GO term ID.
///
/// Raises:
///     ValueError: If the GO term does not exist in the ontology.
#[pyfunction]
pub fn get_term_by_id(go_id: &str) -> PyResult<PyGOTerm> {
    let terms = get_terms_or_error()?;
    match terms.get(go_id) {
        Some(term) => Ok(PyGOTerm::from(term)),
        None => Err(PyValueError::new_err(format!(
            "GO term '{}' not found in ontology",
            go_id
        ))),
    }
}

/// Collect all ancestors of a GO term (recursively via is_a).
///
/// Arguments
/// ---------
/// go_id : str
///   GO term ID.
/// terms : dict
///   Map of GO terms.
///
/// Returns
/// -------
/// HashSet<String>
///   Set of ancestor GO term IDs.
pub fn collect_ancestors(go_id: &str, terms: &HashMap<String, GOTerm>) -> HashSet<String> {
    // Try to use the precomputed cache if available
    if let Some(lock) = crate::go_loader::ANCESTORS_CACHE.get() {
        let cache = lock.read();
        if let Some(ancestors) = cache.get(go_id) {
            return ancestors.clone();
        }
    }
    // Fallback: original computation
    let mut visited = HashSet::default();
    let mut stack = vec![go_id];
    while let Some(current) = stack.pop() {
        if visited.insert(current.to_string()) {
            if let Some(term) = terms.get(current) {
                for parent in &term.parents {
                    stack.push(parent);
                }
            }
        }
    }
//    visited.remove(go_id);
    visited
}

/// Returns the list of all ancestors in the ontology for the given GO Term.
///
/// Arguments
/// ---------
/// go_id : str
///   GO term ID.
///
/// Returns
/// -------
/// list of str
///   List of IDs of all the ancestors in the ontology.
// Esto cuenta como un encapsulamiento del código con la finalidad de que
// solo el usuario provea el GO id.
#[pyfunction]
pub fn ancestors(go_id: &str) -> PyResult<Vec<String>> {
    let terms = get_terms_or_error()?;
    let visited = collect_ancestors(go_id, &terms);
    Ok(visited.into_iter().collect())
}

/// Returns the list of all the common ancestors in the ontology for the given GO Terms.
///
/// Arguments
/// ---------
/// go_id1 : str
///   GO term ID 1.
/// go_id2 : str
///   GO term ID 2.
///
/// Returns
/// -------
/// list of str
///   List of IDs of all the common ancestors in the ontology.
// Esta función lo que realiza es la intersección de todos los términos
// ancestrales entre dos GO ids.
#[pyfunction]
pub fn common_ancestor(go_id1: &str, go_id2: &str) -> PyResult<Vec<String>> {
    let terms = get_terms_or_error()?;
    let set1 = collect_ancestors(go_id1, &terms);
    let set2 = collect_ancestors(go_id2, &terms);
    let mut common: Vec<String> = set1.intersection(&set2).map(|s| (*s).to_string()).collect();
    common.sort_unstable();
    Ok(common)
}

/// Returns the deepest common ancestor in the ontology for the given GO Terms.
///
/// Arguments
/// ---------
/// go_id1 : str
///   GO term ID 1.
/// go_id2 : str
///   GO term ID 2.
///
/// Returns
/// -------
/// Option<String>
///   ID of the deepest common ancestor in the ontology.
#[pyfunction]
pub fn deepest_common_ancestor(go_id1: &str, go_id2: &str) -> PyResult<Option<String>> {
    let terms = get_terms_or_error()?;                                      // Se buscan términos dentro de la cache existente.

    if !terms.contains_key(go_id1) {                                        // Verificación de que los terminos existan dentro del código.
        return Err(PyValueError::new_err(format!(
            "GO term '{}' not found in ontology",
            go_id1
        )));
    }
    if !terms.contains_key(go_id2) {
        return Err(PyValueError::new_err(format!(
            "GO term '{}' not found in ontology",
            go_id2
        )));
    }

    // Se establece el par ordenado de ambos ids, ya que esto estandariza
    // los pares para consulta en cache, como estandarizar que el valor
    // es independiente del orden.
    let (id_a, id_b) = if go_id1 <= go_id2 {
        (go_id1, go_id2)
    } else {
        (go_id2, go_id1)
    };

    // Try to use the DCA cache if available.
    if let Some(lock) = crate::go_loader::DCA_CACHE.get() {
        let cache = lock.write();
        if let Some(result) = cache.get(&(id_a.to_string(), id_b.to_string())) {
            return Ok(Some(result.clone()));
        }
    }

    // En caso de no encontrar ancestro en caché.
    let set1 = collect_ancestors(id_a, &terms);                             // Ancestros del id_a.
    let set2 = collect_ancestors(id_b, &terms);                             // Ancestros del id_b.
    let mut best = None;
    let mut max_depth = 0;
    for term_id in set1.intersection(&set2) {                               // Intersección entre ambas listas de ancestros.
        if let Some(term) = terms.get(term_id) {                            // Se verifican campos depth para obtener el DCA.
            if let Some(depth) = term.depth {
                if depth >= max_depth {
                    max_depth = depth;
                    best = Some(term_id.to_string());
                }
            }
        }
    }

    // Store the result in the cache if available.
    if let Some(lock) = crate::go_loader::DCA_CACHE.get() {
        let mut cache = lock.write();
        if let Some(ref dca) = best {
            cache.insert((id_a.to_string(), id_b.to_string()), dca.clone());
        }
    }

    Ok(best)
}

## Código 4 - semantic.rs

In [ ]:
// -------------------- Librerias 
use pyo3::exceptions::PyValueError;                                             // Se solicita a Python la invocación del ValueError.
use pyo3::prelude::*;                                                           // Rust comprende Python (viceversa).
use rayon::prelude::*;                                                          // Administración de funciones con comportamiento paralelo.                                                  
use crate::go_loader::TermCounter;                                              // Invocación de la estructura de conteo de términos.
use rustc_hash::{FxHashMap as HashMap, FxHashSet as HashSet};                   // Tablas Hash optimizadas.
// Petición de componentes al archivo go_ontology.
use crate::go_ontology::{deepest_common_ancestor, get_term_by_id, get_terms_or_error, get_gene2go_or_error};
use dashmap::DashMap;                                                           // Manejo concurrente de tablas hash.
use rayon::ThreadPoolBuilder;                                                   // Administración de nucleos para trabajo paralelo.

/// Configure the maximum number of threads rayon will use.
///
/// Args:
///     n_threads (int): Number of threads to use. If 0, uses all available cores.
#[pyfunction]
pub fn set_num_threads(n_threads: usize) -> PyResult<()> {
    if n_threads == 0 {
        ThreadPoolBuilder::new()
            .build_global()
            .map_err(|e| pyo3::exceptions::PyRuntimeError::new_err(e.to_string()))?;
    } else {
        ThreadPoolBuilder::new()
            .num_threads(n_threads)
            .build_global()
            .map_err(|e| pyo3::exceptions::PyRuntimeError::new_err(e.to_string()))?;
    }
    Ok(())
}

/// Compute the Information Content (IC) of a GO term.
///
/// Arguments
/// ---------
/// go_id : str
///   GO term identifier.
/// counter : TermCounter
///   Precomputed term counter with IC values.
///
/// Returns
/// -------
/// float
///   The IC of the GO term.
#[pyfunction]
#[pyo3(text_signature = "(go_id, counter)")]
pub fn term_ic(go_id: &str, counter: &TermCounter) -> f64 {
    *counter.ic.get(go_id).unwrap_or(&0.0)
}

// Esta sección de código actua como un menú adaptativo dedicado a la invocación
// de los métodos de similitud entre elementos, incluso, logrando reconocer
// opciones aunque el usuario cometa errores ortograficos.
#[derive(Debug)]
enum SimilarityMethod {
    Resnik,
    Lin,
    JC,
    SimRel,
    ICCoef,
    GraphIC,
    Wang,
    TopoICSim,
}

// Dentro de esta subsección se invocan los distintos tipos de méticas de similitud ya sea
// por medio de términos GO, lo bueno de esta función es que ya existen elementos construidos
// que fueron verificados, ya se puede asegurar que su implementación es correcta.
// Dentro de esta sección solo se implementan otras componentes de protección para establecer el
// resultado como 0.
impl SimilarityMethod {
    fn from_str(name: &str) -> Option<Self> {
        match name.to_lowercase().as_str() {
            "resnik" => Some(SimilarityMethod::Resnik),
            "lin" => Some(SimilarityMethod::Lin),
            "jc" => Some(SimilarityMethod::JC),
            "simrel" => Some(SimilarityMethod::SimRel),
            "iccoef" => Some(SimilarityMethod::ICCoef),
            "graphic" => Some(SimilarityMethod::GraphIC),
            "wang" => Some(SimilarityMethod::Wang),
            "topoicsim" => Some(SimilarityMethod::TopoICSim),
            _ => None,
        }
    }

    fn compute(&self, id1: &str, id2: &str, counter: &TermCounter) -> f64 {
        match self {
            SimilarityMethod::Resnik => {
                let dca = match deepest_common_ancestor(id1, id2).ok().flatten() {
                    Some(dca) => dca,
                    None => return 0.0,
                };
                *counter.ic.get(&dca).unwrap_or(&0.0)
            }
            SimilarityMethod::Lin => {
                let dca = match deepest_common_ancestor(id1, id2).ok().flatten() {
                    Some(dca) => dca,
                    None => return 0.0,
                };
                if id1 == id2 {
                    return 1.0
                }
                let resnik = *counter.ic.get(&dca).unwrap_or(&0.0);
                if resnik == 0.0 {
                    return 0.0;
                }
                let ic1 = *counter.ic.get(id1).unwrap_or(&0.0);
                let ic2 = *counter.ic.get(id2).unwrap_or(&0.0);
                if ic1 == 0.0 || ic2 == 0.0 {
                    return 0.0;
                }
                2.0 * resnik / (ic1 + ic2)
            }
            SimilarityMethod::JC => {
                let (t1, t2) = match (get_term_by_id(id1).ok(), get_term_by_id(id2).ok()) {
                    (Some(t1), Some(t2)) => (t1, t2),
                    _ => return 0.0,
                };
            
                if t1.namespace != t2.namespace {
                    return 0.0;
                }
            
                let ic1 = term_ic(id1, counter);
                let ic2 = term_ic(id2, counter);
            
                let dca_ic = match deepest_common_ancestor(id1, id2).ok().flatten() {
                    Some(dca) => term_ic(&dca, counter),
                    None => return 0.0,
                };
            
                let distance = ic1 + ic2 - 2.0 * dca_ic;
                if distance <= 0.0 {
                    return f64::INFINITY;  // Máxima similitud
                }
                if distance.is_infinite() {
                    0.0
                } else {
                    1.0 / (1.0 + distance)
                }
            }
            SimilarityMethod::SimRel => {
                let (t1, t2) = match (get_term_by_id(id1).ok(), get_term_by_id(id2).ok()) {
                    (Some(t1), Some(t2)) => (t1, t2),
                    _ => return 0.0,
                };
            
                if t1.namespace != t2.namespace {
                    return 0.0;
                }
            
                let ic1 = term_ic(id1, counter);
                let ic2 = term_ic(id2, counter);
            
                if ic1 == 0.0 || ic2 == 0.0 {
                    return 0.0;
                }
            
                let dca_ic = match deepest_common_ancestor(id1, id2).ok().flatten() {
                    Some(dca) => term_ic(&dca, counter),
                    None => return 0.0,
                };
            
                if dca_ic == 0.0 {
                    return 0.0;
                }
            
                let lin = (2.0 * dca_ic) / (ic1 + ic2);
                lin * (1.0 - (-dca_ic).exp())
            }
            SimilarityMethod::ICCoef => {
                let (t1, t2) = match (get_term_by_id(id1).ok(), get_term_by_id(id2).ok()) {
                    (Some(t1), Some(t2)) => (t1, t2),
                    _ => return 0.0,
                };
            
                if t1.namespace != t2.namespace {
                    return 0.0;
                }
            
                let ic1 = term_ic(id1, counter);
                let ic2 = term_ic(id2, counter);
            
                if ic1 == 0.0 || ic2 == 0.0 {
                    return 0.0;
                }
            
                let dca_ic = match deepest_common_ancestor(id1, id2).ok().flatten() {
                    Some(dca) => term_ic(&dca, counter),
                    None => return 0.0,
                };
            
                dca_ic / ic1.min(ic2)
            }
            SimilarityMethod::GraphIC => {
                let (t1, t2) = match (get_term_by_id(id1).ok(), get_term_by_id(id2).ok()) {
                    (Some(t1), Some(t2)) => (t1, t2),
                    _ => return 0.0,
                };
            
                if t1.namespace != t2.namespace {
                    return 0.0;
                }
            
                let depth1 = t1.depth.unwrap_or(0);
                let depth2 = t2.depth.unwrap_or(0);
                let max_depth = (depth1.max(depth2) + 1) as f64;
            
                let dca_ic = match deepest_common_ancestor(id1, id2).ok().flatten() {
                    Some(dca) => term_ic(&dca, counter),
                    None => return 0.0,
                };
            
                dca_ic / max_depth
            }
            SimilarityMethod::Wang => {
                let terms = match crate::go_loader::GO_TERMS_CACHE.get() {
                    Some(lock) => lock.read(),
                    None => return 0.0,
                };
            
                let terms = &*terms;
            
                let t1 = match terms.get(id1) {
                    Some(t) => t,
                    None => return 0.0,
                };
                let t2 = match terms.get(id2) {
                    Some(t) => t,
                    None => return 0.0,
                };
            
                if t1.namespace != t2.namespace {
                    return 0.0;
                }
            
                let sv_a = semantic_contributions(id1, terms);
                let sv_b = semantic_contributions(id2, terms);
            
                let sum_a: f64 = sv_a.values().sum();
                let sum_b: f64 = sv_b.values().sum();
            
                let common_keys: std::collections::HashSet<_> = sv_a.keys().collect::<HashSet<_>>()
                    .intersection(&sv_b.keys().collect::<HashSet<_>>())
                    .cloned()
                    .collect();
            
                let mut numerator = 0.0;
                for key in common_keys {
                    if let (Some(w1), Some(w2)) = (sv_a.get(key), sv_b.get(key)) {
                        numerator += (*w1).min(*w2);
                    }
                }
            
                if sum_a + sum_b == 0.0 {
                    0.0
                } else {
                    numerator / ((sum_a + sum_b) / 2.0)
                }
            }
            SimilarityMethod::TopoICSim => {
                // Get terms and check namespace
                let terms = match crate::go_loader::GO_TERMS_CACHE.get() {
                    Some(lock) => lock.read(),
                    None => return 0.0,
                };
                let t1 = match terms.get(id1) {
                    Some(t) => t,
                    None => return 0.0,
                };
                let t2 = match terms.get(id2) {
                    Some(t) => t,
                    None => return 0.0,
                };
                if t1.namespace != t2.namespace {
                    return 0.0;
                }
                // Disjunctive common ancestors
                let dca_set = disjunctive_common_ancestors(id1, id2, &terms);
                if dca_set.is_empty() {
                    return 0.0;
                }
                // Find all roots
                let roots = find_roots(&terms);
                if roots.is_empty() {
                    return 0.0;
                }
                let mut min_d = f64::INFINITY;
                for x in dca_set {
                    // Weighted shortest path from t1 to x and t2 to x
                    let wsp1 = weighted_shortest_path_iic(id1, &x, &terms, counter);
                    let wsp2 = weighted_shortest_path_iic(id2, &x, &terms, counter);
                    if wsp1.is_none() || wsp2.is_none() {
                        continue;
                    }
                    let wsp = wsp1.unwrap() + wsp2.unwrap();
                    // Weighted longest path from x to any root (take the max over all roots)
                    let mut max_wlp = None;
                    for root in &roots {
                        if let Some(wlp) = weighted_longest_path_iic(&x, root, &terms, counter) {
                            max_wlp = Some(max_wlp.map_or(wlp, |m: f64| m.max(wlp)));
                        }
                    }
                    let wlp = match max_wlp {
                        Some(val) if val > 0.0 => val,
                        _ => continue,
                    };
                    let d = wsp / wlp;
                    if d < min_d {
                        min_d = d;
                    }
                }
                if !min_d.is_finite() {
                    return 0.0;
                }
                // Similarity formula: 1 - (arctan(D) / (pi/2))
                let sim = 1.0 - (min_d.atan() / (std::f64::consts::FRAC_PI_2));
                if sim.is_finite() && sim > 0.0 { sim } else { 0.0 }
            }
        }
    }
}

/// Compute semantic similarity between two GO terms using a selected method.
///
/// Arguments
/// ---------
/// id1 : str
///   First GO term ID.
/// id2 : str
///   Second GO term ID.
/// method : str
///   Name of the similarity method. Options: "resnik", "lin", etc.
/// counter : TermCounter
///   Precomputed IC values.
///
/// Returns
/// -------
/// float
///   Similarity score.
///
/// Raises
/// ------
/// ValueError
///   If the method is unknown.
#[pyfunction]
pub fn semantic_similarity(
    id1: &str,
    id2: &str,
    method: &str,
    counter: &TermCounter,
) -> PyResult<f64> {

    let method_enum = SimilarityMethod::from_str(method)
        .ok_or_else(|| PyValueError::new_err(format!("Unknown similarity method: {}", method)))?;

    Ok(method_enum.compute(id1, id2, counter))
}

/// Compute pairwise semantic similarity in batch using a selected method.
///
/// Arguments
/// ---------
/// list1 : list of str
///   First list of GO term IDs.
/// list2 : list of str
///   Second list of GO term IDs.
/// method : str
///   Name of the similarity method.
/// counter : TermCounter
///   Precomputed IC values.
///
/// Returns
/// -------
/// list of float
///   List of similarity scores.
///
/// Raises
/// ------
/// ValueError
///   If input lists differ in length or method is unknown.
#[pyfunction]
pub fn batch_similarity(
    list1: Vec<String>,
    list2: Vec<String>,
    method: &str,
    counter: &TermCounter,
) -> PyResult<Vec<f64>> {
    if list1.len() != list2.len() {
        return Err(PyValueError::new_err("Both lists must be the same length"));
    }

    let method_enum = SimilarityMethod::from_str(method)
        .ok_or_else(|| PyValueError::new_err(format!("Unknown similarity method: {}", method)))?;

    // 1. Collect all unique pairs (order them to avoid (a,b) vs (b,a) duplicates)
    let unique_pairs: HashSet<(String, String)> = list1.iter().zip(list2.iter())
        .map(|(a, b)| {
            if a <= b {
                (a.clone(), b.clone())
            } else {
                (b.clone(), a.clone())
            }
        })
        .collect();

    // 2. Compute similarity for each unique pair in parallel
    let sim_map: HashMap<(String, String), f64> = unique_pairs
        .par_iter()
        .map(|(a, b)| {
            let sim = method_enum.compute(a, b, counter);
            ((a.clone(), b.clone()), sim)
        })
        .collect();

    // 3. For each original pair, look up the result (parallelized)
    let result: Vec<f64> = list1.par_iter().zip(list2.par_iter())
        .map(|(a, b)| {
            let key = if a <= b { (a.clone(), b.clone()) } else { (b.clone(), a.clone()) };
            *sim_map.get(&key).unwrap_or(&0.0)
        })
        .collect();

    Ok(result)
}


/// Compute semantic similarity between genes.
///
/// Arguments
/// ---------
/// gene1 : str
///   Gene symbol of the first gene.
/// gene2 : str
///   Gene symbol of the second gene.
/// ontology : str
///   Name of the subontology of GO to use: BP, MF or CC.
/// similarity : str
///   Name of the similarity method.
/// groupwise : str
///   Combination method to generate the similarities between genes. Options: "bma", "max".
/// counter : TermCounter
///   Precomputed IC values.
///
/// Returns
/// -------
/// float
///   Similarity score.
///
/// Raises
/// ------
/// ValueError
///   If method or combine are unknown.
#[pyfunction]
pub fn compare_genes(
    gene1: &str,
    gene2: &str,
    ontology: String,
    similarity: &str,
    groupwise: String,
    counter: &TermCounter,
) -> PyResult<f64> {
    let terms = get_terms_or_error()?;
    let gene2go = get_gene2go_or_error()?;
    let g1_terms = gene2go.get(gene1).ok_or_else(|| {
        pyo3::exceptions::PyValueError::new_err(format!("Gene '{}' not found in mapping", gene1))
    })?;
    let g2_terms = gene2go.get(gene2).ok_or_else(|| {
        pyo3::exceptions::PyValueError::new_err(format!("Gene '{}' not found in mapping", gene2))
    })?;
    let ns = match ontology.as_str() {
        "BP" => "biological_process",
        "MF" => "molecular_function",
        "CC" => "cellular_component",
        _ => {
            return Err(PyValueError::new_err(format!(
                "Invalid ontology '{}'. Must be 'BP', 'MF', or 'CC'",
                ontology
            )))
        }
    };
    let f1: Vec<String> = g1_terms
        .iter()
        .filter(|id| terms.get(*id).map_or(false, |t| t.namespace.to_ascii_lowercase() == ns))
        .cloned()
        .collect();

    let f2: Vec<String> = g2_terms
        .iter()
        .filter(|id| terms.get(*id).map_or(false, |t| t.namespace.to_ascii_lowercase() == ns))
        .cloned()
        .collect();
    print!("{:?}", f1);
    print!("{:?}", f2);
    if f1.is_empty() || f2.is_empty() {
        return Ok(0.0);
    }

    let sim_fn = SimilarityMethod::from_str(similarity)
        .ok_or_else(|| PyValueError::new_err(format!("Unknown similarity method: {}", similarity)))?;

    let score = match groupwise.as_str() {
        "max" => {
            f1.par_iter()
                .map(|id1| {
                    f2.par_iter()
                        .map(|id2| sim_fn.compute(id1, id2, counter))
                        .reduce(|| 0.0, f64::max)
                })
                .reduce(|| 0.0, f64::max)
        }
        "bma" => {
            let sem1: Vec<f64> = f1.par_iter()
                .map(|id1| {
                    f2.par_iter()
                        .map(|id2| sim_fn.compute(id1, id2, counter))
                        .reduce(|| 0.0, f64::max)
                })
                .collect();

            let sem2: Vec<f64> = f2.par_iter()
                .map(|id2| {
                    f1.par_iter()
                        .map(|id1| sim_fn.compute(id1, id2, counter))
                        .reduce(|| 0.0, f64::max)
                })
                .collect();

            let total = sem1.len() + sem2.len();
            if total == 0 {
                0.0
            } else {
                (sem1.iter().sum::<f64>() + sem2.iter().sum::<f64>()) / total as f64
            }
        }
        _ => return Err(pyo3::exceptions::PyValueError::new_err("Unknown groupwise strategy")),
    };

    Ok(score)
}

/// Compute semantic similarity between genes in batches.
///
/// Arguments
/// ---------
/// pairs : list of (str, str)
///   List of pairs of genes to calculate the semantic similarity
/// ontology : str
///   Name of the subontology of GO to use: BP, MF or CC.
/// similarity : str
///   Name of the similarity method.
/// groupwise : str
///   Combination method to generate the similarities between genes. Options: "bma", "max".
/// counter : TermCounter
///   Precomputed IC values.
///
/// Returns
/// -------
/// list of float
///   List of similarity scores.
///
/// Raises
/// ------
/// ValueError
///   If method or combine are unknown.
#[pyfunction]
#[pyo3(signature = (pairs, ontology, similarity, groupwise, counter))]
pub fn compare_gene_pairs_batch(
    pairs: Vec<(String, String)>,
    ontology: String,
    similarity: &str,
    groupwise: String,
    counter: &TermCounter,
) -> PyResult<Vec<f64>> {
    let gene2go = get_gene2go_or_error()?;
    let terms = get_terms_or_error()?;

    let ns = match ontology.as_str() {
        "BP" => "biological_process",
        "MF" => "molecular_function",
        "CC" => "cellular_component",
        _ => {
            return Err(PyValueError::new_err(format!(
                "Invalid ontology '{}'. Must be 'BP', 'MF', or 'CC'",
                ontology
            )))
        }
    };

    let sim_fn = SimilarityMethod::from_str(similarity)
        .ok_or_else(|| PyValueError::new_err(format!("Unknown similarity method: {}", similarity)))?;

    let scores: Vec<f64> = pairs
        .into_par_iter()
        .map(|(g1, g2)| {
            let go1: Vec<_> = gene2go
                .get(&g1)
                .into_iter()
                .flatten()
                .filter(|go| terms.get(go.as_str()).map_or(false, |t| t.namespace.eq_ignore_ascii_case(ns)))
                .cloned()
                .collect();

            let go2: Vec<_> = gene2go
                .get(&g2)
                .into_iter()
                .flatten()
                .filter(|go| terms.get(go.as_str()).map_or(false, |t| t.namespace.eq_ignore_ascii_case(ns)))
                .cloned()
                .collect();

            if go1.is_empty() || go2.is_empty() {
                return 0.0;
            }

            match groupwise.as_str() {
                "max" => go1.par_iter()
                    .map(|id1| go2.par_iter().map(|id2| sim_fn.compute(id1, id2, counter)).reduce(|| 0.0, f64::max))
                    .reduce(|| 0.0, f64::max),
                "bma" => {
                    let sem1: Vec<_> = go1.par_iter()
                        .map(|id1| go2.par_iter().map(|id2| sim_fn.compute(id1, id2, counter)).reduce(|| 0.0, f64::max))
                        .collect();
                    let sem2: Vec<_> = go2.par_iter()
                        .map(|id2| go1.par_iter().map(|id1| sim_fn.compute(id1, id2, counter)).reduce(|| 0.0, f64::max))
                        .collect();
                    let total = sem1.len() + sem2.len();
                    if total == 0 {
                        0.0
                    } else {
                        (sem1.iter().sum::<f64>() + sem2.iter().sum::<f64>()) / total as f64
                    }
                }
                _ => 0.0,
            }
        })
        .collect();

    Ok(scores)
}

fn semantic_contributions(
    go_id: &str,
    terms: &HashMap<String, crate::go_ontology::GOTerm>,
) -> HashMap<String, f64> {
    // Static cache for memoization
    static SEMANTIC_CONTRIB_CACHE: once_cell::sync::Lazy<DashMap<String, HashMap<String, f64>>> = once_cell::sync::Lazy::new(|| DashMap::new());

    // Check cache first
    if let Some(cached) = SEMANTIC_CONTRIB_CACHE.get(go_id) {
        return cached.clone();
    }

    let mut contributions = HashMap::default();
    let mut to_visit = vec![(go_id, 1.0)];

    while let Some((current_id, weight)) = to_visit.pop() {
        if weight < 1e-6 || contributions.contains_key(current_id) {
            continue;
        }

        contributions.insert(current_id.to_string(), weight);

        if let Some(term) = terms.get(current_id) {
            // is_a → 0.8
            for parent in &term.parents {
                to_visit.push((parent, weight * 0.8));
            }
            // part_of → 0.6
            for (rel_type, target) in &term.relationships {
                let rel_weight = match rel_type.as_str() {
                    "part_of" => 0.6,
                    _ => continue,  // skip other relationships for now
                };
                to_visit.push((target, weight * rel_weight));
            }
        }
    }

    // Store in cache as HashMap<String, f64>
    SEMANTIC_CONTRIB_CACHE.insert(go_id.to_string(), contributions.clone());
    contributions
}

// --- TopoICSim and helper functions ---

/// Compute the Inverse Information Content (IIC) for a term.
///
/// Arguments
/// ---------
/// go_id : str
///   GO term ID.
/// counter : TermCounter
///   Precomputed term counter with IC values.
///
/// Returns
/// -------
/// float
///   The IIC value for the term.
fn iic(go_id: &str, counter: &TermCounter) -> f64 {
    let ic = *counter.ic.get(go_id).unwrap_or(&0.0);
    if ic > 0.0 {
        1.0 / ic
    } else {
        // If IC is zero, treat as very large (effectively infinite path weight)
        1e12
    }
}

/// Compute the weighted shortest path (sum of IICs) from source to target (ancestor) in the GO DAG.
///
/// Arguments
/// ---------
/// source : str
///   Source GO term ID.
/// target : str
///   Target GO term ID (ancestor).
/// terms : dict
///   Map of GO terms.
/// counter : TermCounter
///   Precomputed term counter with IC values.
///
/// Returns
/// -------
/// Option<float>
///   Minimum sum of IICs along any path from source to target, or None if not connected.
fn weighted_shortest_path_iic(
    source: &str,
    target: &str,
    terms: &HashMap<String, crate::go_ontology::GOTerm>,
    counter: &TermCounter,
) -> Option<f64> {
    use std::collections::{BinaryHeap, HashMap};
    use std::cmp::Ordering;
    #[derive(Copy, Clone, PartialEq)]
    struct State {
        cost: f64,
        node: usize,
    }
    impl Eq for State {}
    impl Ord for State {
        fn cmp(&self, other: &Self) -> Ordering {
            // Reverse order for min-heap
            other.cost.partial_cmp(&self.cost).unwrap_or(Ordering::Equal)
        }
    }
    impl PartialOrd for State {
        fn partial_cmp(&self, other: &Self) -> Option<Ordering> {
            Some(self.cmp(other))
        }
    }
    // Map node ids to indices for fast lookup
    let mut id2idx = HashMap::new();
    let mut idx2id = Vec::new();
    for (i, id) in terms.keys().enumerate() {
        id2idx.insert(id.as_str(), i);
        idx2id.push(id.as_str());
    }
    let src = match id2idx.get(source) {
        Some(&i) => i,
        None => return None,
    };
    let tgt = match id2idx.get(target) {
        Some(&i) => i,
        None => return None,
    };
    let mut dist = vec![f64::INFINITY; idx2id.len()];
    dist[src] = iic(source, counter);
    let mut heap = BinaryHeap::new();
    heap.push(State { cost: dist[src], node: src });
    while let Some(State { cost, node }) = heap.pop() {
        let node_id = idx2id[node];
        if node == tgt {
            return Some(cost);
        }
        if cost > dist[node] {
            continue;
        }
        if let Some(term) = terms.get(node_id) {
            for parent in &term.parents {
                if let Some(&parent_idx) = id2idx.get(parent.as_str()) {
                    let next = cost + iic(parent, counter);
                    if next < dist[parent_idx] {
                        dist[parent_idx] = next;
                        heap.push(State { cost: next, node: parent_idx });
                    }
                }
            }
        }
    }
    None
}

/// Compute the weighted longest path (sum of IICs) from source to target (ancestor) in the GO DAG.
///
/// Arguments
/// ---------
/// source : str
///   Source GO term ID.
/// target : str
///   Target GO term ID (ancestor).
/// terms : dict
///   Map of GO terms.
/// counter : TermCounter
///   Precomputed term counter with IC values.
///
/// Returns
/// -------
/// Option<float>
///   Maximum sum of IICs along any path from source to target, or None if not connected.
fn weighted_longest_path_iic(
    source: &str,
    target: &str,
    terms: &HashMap<String, crate::go_ontology::GOTerm>,
    counter: &TermCounter,
) -> Option<f64> {
    // Memoization
    use std::collections::HashMap as StdHashMap;
    fn dfs(
        node: &str,
        target: &str,
        terms: &HashMap<String, crate::go_ontology::GOTerm>,
        counter: &TermCounter,
        memo: &mut StdHashMap<String, Option<f64>>,
    ) -> Option<f64> {
        if node == target {
            return Some(iic(node, counter));
        }
        if let Some(&val) = memo.get(node) {
            return val;
        }
        let mut max_path = None;
        if let Some(term) = terms.get(node) {
            for parent in &term.parents {
                if let Some(sub) = dfs(parent, target, terms, counter, memo) {
                    let total = iic(node, counter) + sub;
                    max_path = Some(max_path.map_or(total, |m: f64| m.max(total)));
                }
            }
        }
        memo.insert(node.to_string(), max_path);
        max_path
    }
    let mut memo = StdHashMap::new();
    dfs(source, target, terms, counter, &mut memo)
}

/// Compute the Disjunctive Common Ancestor set for two terms (as per TopoICSim paper).
///
/// Arguments
/// ---------
/// id1 : str
///   First GO term ID.
/// id2 : str
///   Second GO term ID.
/// terms : dict
///   Map of GO terms.
///
/// Returns
/// -------
/// list of str
///   List of disjunctive common ancestor GO term IDs.
fn disjunctive_common_ancestors(
    id1: &str,
    id2: &str,
    terms: &HashMap<String, crate::go_ontology::GOTerm>,
) -> Vec<String> {
    use std::collections::HashSet;
    // 1. Get all common ancestors
    let ancestors1 = crate::go_ontology::collect_ancestors(id1, terms);
    let ancestors2 = crate::go_ontology::collect_ancestors(id2, terms);
    let common: HashSet<_> = ancestors1.intersection(&ancestors2).cloned().collect();
    // 2. For each x in common, check if no child of x is also in common
    let mut dca = Vec::new();
    for x in &common {
        let is_disjunctive = terms.get(x).map_or(false, |term| {
            term.children.iter().all(|c| !common.contains(c))
        });
        if is_disjunctive {
            dca.push(x.clone());
        }
    }
    dca
}

/// Find all root terms (terms with no parents) in the ontology.
///
/// Arguments
/// ---------
/// terms : dict
///   Map of GO terms.
///
/// Returns
/// -------
/// list of str
///   List of root GO term IDs.
fn find_roots(terms: &HashMap<String, crate::go_ontology::GOTerm>) -> Vec<String> {
    terms.iter()
        .filter(|(_, term)| term.parents.is_empty())
        .map(|(id, _)| id.clone())
        .collect()
}

## ¿Por qué varían los resultados de R y GoSemSim?
